In [ ]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import models
from torchvision import transforms
from collections import OrderedDict
from collections import defaultdict
from torchsummary import summary
from torch import nn
from torch.quantization import quantize_fx
import copy


torch.set_printoptions(profile="full")
import numpy as np
np.set_printoptions(threshold=np.inf)
#torch.set_default_dtype(torch.quint8)

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/Colab Notebooks')
cwd= os.getcwd()
print (cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
path = "/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats"
os.chdir(path)
train_df = pd.DataFrame(columns=["img_name","label"])
train_df["img_name"] = os.listdir("train/")
for idx, i in enumerate(os.listdir("train/")):
    if "cat" in i:
        train_df["label"][idx] = 0
    if "dog" in i:
        train_df["label"][idx] = 1

train_df.to_csv (r'train_csv.csv', index = False, header=True)

In [ ]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [ ]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [ ]:
# First, load the model


resnet = models.resnet34(pretrained=True)
resnet.name = 'resnet'
alexnet = models.alexnet(pretrained=True)
alexnet.name = 'alexnet'
vgg19 = models.vgg19(pretrained=True)
vgg19.name = 'vgg19'


models_list = [resnet]



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weight

In [ ]:
transform = transforms.Compose([
 transforms.Resize(256),
 transforms.CenterCrop(224),
 transforms.ToTensor(),
 transforms.Normalize(
 mean=[0.485, 0.456, 0.406],
 std=[0.229, 0.224, 0.225]
 )])

In [ ]:
for model in models_list:
  print(model)
  model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
with open(os.path.join("/content/drive/MyDrive/Colab Notebooks",'imagenet_classes.txt')) as f:
  labels = [line.strip() for line in f.readlines()]

In [ ]:
#img=Image.open(os.path.join("/content/drive/MyDrive/Colab Notebooks", "dog.jpg"))

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats/test1"
def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(os.path.join(path, image_name))
    img_t = transform(image)
    batch_t1 = torch.unsqueeze(img_t, 0)
    return batch_t1

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats/test1"
os.chdir(path)
cwd= os.getcwd()
print(cwd)

/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats/test1


In [ ]:
def rep_dataset():
    """Generator function to produce representative dataset for post-training quantization."""

    # Use a few samples from the training set.
    for _ in range(100):
        img = iter(train_loader).next()[0].numpy()
        img = [(img.astype(np.float32))]
        yield img

In [ ]:
# define a floating point model where some layers could be statically quantized
class M(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # QuantStub converts tensors from floating point to quantized
        self.quant = torch.ao.quantization.QuantStub()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.relu = torch.nn.ReLU()
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        # manually specify where tensors will be converted from floating
        # point to quantized in the quantized model
        x = self.quant(x)
        x = self.conv(x)
        x = self.relu(x)
        # manually specify where tensors will be converted from quantized
        # to floating point in the quantized model
        x = self.dequant(x)
        return x

# create a model instance
model_fp32 = M()

# model must be set to eval mode for static quantization logic to work
model_fp32.eval()

# attach a global qconfig, which contains information about what kind
# of observers to attach. Use 'x86' for server inference and 'qnnpack'
# for mobile inference. Other quantization configurations such as selecting
# symmetric or asymmetric quantization and MinMax or L2Norm calibration techniques
# can be specified here.
# Note: the old 'fbgemm' is still available but 'x86' is the recommended default
# for server inference.
# model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('fbgemm')
model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('x86')

# Fuse the activations to preceding layers, where applicable.
# This needs to be done manually depending on the model architecture.
# Common fusions include `conv + relu` and `conv + batchnorm + relu`
model_fp32_fused = torch.ao.quantization.fuse_modules(model_fp32, [['conv', 'relu']])

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
model_fp32_prepared = torch.ao.quantization.prepare(model_fp32_fused)

# calibrate the prepared model to determine quantization parameters for activations
# in a real world setting, the calibration would be done with a representative dataset
input_fp32 = torch.randn(4, 1, 4, 4)
model_fp32_prepared(input_fp32)

# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, and replaces key operators with quantized
# implementations.
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8
res = model_int8(input_fp32)

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
models_quantized_list = []
for model in models_list:
  res_model = M()
  print(res_model)
  res_model.name = model.name + '_quantized'
  models_quantized_list.append(res_model)


M(
  (quant): QuantStub()
  (conv): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (dequant): DeQuantStub()
)


In [ ]:
def create_hooks(model_name,layer_name):
  selected_out = OrderedDict()
  selected_input = OrderedDict()
  return [selected_out, selected_input]

def forward_hook(layer_name, inputstr, outputstr):
        def hook(module, input, output):
            #selected_out[layer_name] = output
            #selected_input[layer_name] = input
            data[inputstr][layer_name] = input
            data[outputstr][layer_name] = output

            #df2 = {'layer': 'layer_name', 'input_size': input[0].size(), 'output_size': output.data.size(), 'output_norm':output.data.norm()}

        return hook


fhooks = []

data = defaultdict(dict)
for model in models_quantized_list:
  for i in ['input', 'output']:
    data["_".join([model.name, i])] = OrderedDict()

for model in models_list:
  for name, layer in model._modules.items():
    print(name)
    #inputarr, outputarr = create_hooks(model.name, name)
    inputstr = model.name + '_' + 'input'
    outputstr = model.name + '_' + 'output'

    fhooks.append(layer.register_forward_hook(forward_hook(name, inputstr, outputstr)))

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats"
os.chdir(path)

In [ ]:
num_epochs = 10
learning_rate = 0.00001
train_CNN = False
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 0
dataset = CatsAndDogsDataset("train","train_csv.csv",transform=transform)
print(len(dataset))
#train_size = int(0.8 * len(dataset))
#train_set= torch.utils.data.random_split(dataset, [train_size])
train_loader = DataLoader(dataset=dataset, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)



10601


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats/test1"
os.chdir(path)
cwd= os.getcwd()
print(cwd)

/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats/test1


In [ ]:
def runtest(model):
  for idx, i in enumerate(os.listdir(cwd)[:10] ):
    batch_t = image_loader(i)
    out = model(batch_t)
    print(out.shape)
    _, index = torch.max(out, 1)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    print(i, labels[index[0]], percentage[index[0]].item())

for model in models_list:
  print('running test for model list ', model.name)
  runtest(model)

running test for model list  resnet
torch.Size([1, 1000])
11455.jpg tabby, tabby cat 57.0185432434082
torch.Size([1, 1000])
11459.jpg Chihuahua 97.77523803710938
torch.Size([1, 1000])
11564.jpg Egyptian cat 60.01301574707031
torch.Size([1, 1000])
11505.jpg lynx, catamount 60.65275192260742
torch.Size([1, 1000])
11514.jpg whippet 22.04637336730957
torch.Size([1, 1000])
11535.jpg Rhodesian ridgeback 88.1002426147461
torch.Size([1, 1000])
11546.jpg Egyptian cat 30.778223037719727
torch.Size([1, 1000])
11504.jpg Egyptian cat 27.995555877685547
torch.Size([1, 1000])
11561.jpg Shetland sheepdog, Shetland sheep dog, Shetland 45.047874450683594
torch.Size([1, 1000])
11508.jpg tabby, tabby cat 67.6646499633789


In [ ]:
import csv
from torch.fx import symbolic_trace

for key, value in data.items():
  dir_path = "/content/drive/MyDrive/Colab Notebooks/" + key
  if not os.path.isdir(dir_path):
    os.makedirs(dir_path)
  print("model is ", key)
  for k1, val1 in data[key].items():
    file_name = k1 + '.csv'
    print("layer is ", k1)
    try:

      with open(os.path.join(dir_path,file_name), "w") as csv_file:
        csvwriter = csv.writer(csv_file)
        csvwriter.writerow(val1)

    except Exception as e:
      substring = "TraceError"
      if substring in str(e):
        print("Proxy Object Error, continuing")
      else:
        print(e)


model is  resnet_quantized_input
model is  resnet_quantized_output
model is  resnet_input
layer is  conv1
layer is  bn1
layer is  relu
layer is  maxpool
layer is  layer1
layer is  layer2
layer is  layer3
layer is  layer4
layer is  avgpool
layer is  fc
model is  resnet_output
layer is  conv1
layer is  bn1
layer is  relu
layer is  maxpool
layer is  layer1
layer is  layer2
layer is  layer3
layer is  layer4
layer is  avgpool
layer is  fc


In [ ]:



# Forth, print the top 5 classes predicted by the model
#_, indices = torch.sort(out, descending=True)
#percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
#[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

In [ ]:
for model in models_list:
  summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:

for model in models_list:
  for name,layer in model.named_children():
    weights = list(layer.parameters())
    print("Layer: ",name)
    weights_path = "/content/drive/MyDrive/Colab Notebooks/" + model.name
    if not os.path.isdir(weights_path):
      os.makedirs(weights_path)
    file_name = name + '_weights.csv'
    with open(os.path.join(weights_path,file_name), "w") as csv_file:
      print("Writing to model ", model.name)
      csvwriter = csv.writer(csv_file)
      csvwriter.writerow(weights)


Layer:  conv1
Writing to model  resnet
Layer:  bn1
Writing to model  resnet
Layer:  relu
Writing to model  resnet
Layer:  maxpool
Writing to model  resnet
Layer:  layer1
Writing to model  resnet
Layer:  layer2
Writing to model  resnet
Layer:  layer3
Writing to model  resnet
Layer:  layer4
Writing to model  resnet
Layer:  avgpool
Writing to model  resnet
Layer:  fc
Writing to model  resnet
